# Locating restriction sites, from [Rosalind.info](https://www.rosalind.info)

(Specific exercise can be found at: http://rosalind.info/problems/revp/)

## My personal interpretation of the problem

1. The problem is about DNA palindromes (or possible restriction sites)

2. The input will be a DNA sequence of at most 1 kbp long

3. What the program than needs to do is read the sequence from start do end,

    - while checking for each substring of length 2-6 (the forward half of the potential 4-12 bp palindrome) if the next substring of equal length is its reverse complement
    
4. The output should then be 1) the position of this site, and 2) the length of the palindrome

    - so not only keep the nucleotides, but keep the position and lengths (too)!
    

To tackle this problem, I want to make functions that:

1. Read a sequence from a fasta file

2. Find palindromes! (includes: read sequence, check reverse complements of substrings, keep positions)

In [2]:
from Bio.Seq import Seq #for working with Seq objects and making reverse complements
from Bio import SeqIO #for reading fasta files
from pathlib import Path #for working with files in general

In [21]:
#Set minimum and maximum lengths of palindrome halves
min_length = 2
max_length = 6

In [3]:
def read_fasta_file(input_file):
    """
Given an input fasta file, read the fasta record (ID and sequence).
(Note: read only one sequence!)
    """
    for seq_record in SeqIO.parse(input_file, "fasta"):
        name = seq_record.id #this is not necessary in this exercise
        sequence = seq_record.seq
        break
    
    return name, sequence

In [4]:
example_file = Path("data/Example_Locating_restriction_sites.fasta")

(example_name, example_sequence) = read_fasta_file(example_file)

print(example_sequence)

TCAATGCATGCGGGTCTATATGCAT


Alright, the fasta reading function seems to be working. Now move on to the palindrome finder.  
...  
After doing some quick tests with the `.reverse_complement()` method!

In [5]:
print(example_sequence.reverse_complement())

ATGCATATAGACCCGCATGCATTGA


In [6]:
print(example_sequence[3:8].reverse_complement())

TGCAT


Right, that works as I expected. Now let's try and put that into the palindrome finder function.

In [7]:
def find_DNA_palindromes(sequence):
    """
Given a DNA sequence, find all palindromes of lengths 4-12
and report their position and length, and include the sequence
for easier debugging.

Save palindromes as a dictionary, using positions as key and
length and sequence as values (as tuple).
    """
    palindrome_dict = {}
    
    for position in range(len(sequence)):
        for length in range(2,6):
            potential_forward_half = sequence[position:position+length+1]
            potential_reverse_half = sequence[position+length+1:position+(length*2)]
            
            print("Length: %i, forward: %s, reverse: %s" % (
            length,
            potential_forward_half,
            potential_reverse_half
            )
                 )
            
            
            if potential_forward_half == potential_reverse_half.reverse_complement():
                #This is a palindrome!
                palindrome_dict[position] = (
                    length, 
                    potential_forward_half + potential_reverse_half
                )
            else:
                pass
            
        return palindrome_dict
            

In [8]:
example_palindromes = find_DNA_palindromes(example_sequence)

Length: 2, forward: TCA, reverse: A
Length: 3, forward: TCAA, reverse: TG
Length: 4, forward: TCAAT, reverse: GCA
Length: 5, forward: TCAATG, reverse: CATG


Right, so this does not seem to be working correctly yet.
The forward sequence starts as one too long,
the reverse sequence is too short,
and the palindromes are returned after checking one position.

Let's see if I can fix those errors:

In [9]:
def find_DNA_palindromes(sequence):
    """
Given a DNA sequence, find all palindromes of lengths 4-12
and report their position and length, and include the sequence
for easier debugging.

Save palindromes as a dictionary, using positions as key and
length and sequence as values (as tuple).
    """
    palindrome_dict = {}
    
    for position in range(len(sequence) + 1):
        for length in range(2,6):
            potential_forward_half = sequence[position:position+length]
            potential_reverse_half = sequence[position+length+1:position+(length*2)+1]
            
            print("Length: %i, forward: %s, reverse: %s" % (
            length,
            potential_forward_half,
            potential_reverse_half
            )
                 )
            
            
            if potential_forward_half == potential_reverse_half.reverse_complement():
                #This is a palindrome!
                print("%s and %s appear to form a palindrome!" % (
                potential_forward_half,
                potential_reverse_half
                )
                     )
                
                palindrome_dict[position] = (
                    length, 
                    potential_forward_half + potential_reverse_half
                )
            else:
                pass
            
    return palindrome_dict

In [10]:
example_palindromes = find_DNA_palindromes(example_sequence)

Length: 2, forward: TC, reverse: AT
Length: 3, forward: TCA, reverse: TGC
Length: 4, forward: TCAA, reverse: GCAT
Length: 5, forward: TCAAT, reverse: CATGC
Length: 2, forward: CA, reverse: TG
CA and TG appear to form a palindrome!
Length: 3, forward: CAA, reverse: GCA
Length: 4, forward: CAAT, reverse: CATG
Length: 5, forward: CAATG, reverse: ATGCG
Length: 2, forward: AA, reverse: GC
Length: 3, forward: AAT, reverse: CAT
Length: 4, forward: AATG, reverse: ATGC
Length: 5, forward: AATGC, reverse: TGCGG
Length: 2, forward: AT, reverse: CA
Length: 3, forward: ATG, reverse: ATG
Length: 4, forward: ATGC, reverse: TGCG
Length: 5, forward: ATGCA, reverse: GCGGG
Length: 2, forward: TG, reverse: AT
Length: 3, forward: TGC, reverse: TGC
Length: 4, forward: TGCA, reverse: GCGG
Length: 5, forward: TGCAT, reverse: CGGGT
Length: 2, forward: GC, reverse: TG
Length: 3, forward: GCA, reverse: GCG
Length: 4, forward: GCAT, reverse: CGGG
Length: 5, forward: GCATG, reverse: GGGTC
Length: 2, forward: CA, r

Right, I need a few more checks in here:

1. it may be nice to print the position as well

2. the function needs to stop when at the end of the sequence (e.g. not try and make palindromes of non-existing/empty sequences!)

Also, the reverse sequence seems to start one too late.

In [28]:
def find_DNA_palindromes(sequence, debug=False):
    """
Given a DNA sequence, find all palindromes of lengths 4-12
and report their position and length, and include the sequence
for easier debugging.

Save palindromes as a dictionary, using positions as key and
length and sequence as values (as tuple).
    """
    #initialise empty dictionary,
    # to be filled with the palindromes
    palindrome_dict = {}
    
    #loop over the complete sequence, by positions
    for position in range(len(sequence) + 1):
        #then for each position, check for potential palindromes
        for length in range(min_length, max_length + 1):
            
            if len(sequence[position:position+length]) == length:
                #Check if the substring to check is as long as intended,
                # i.e. does not 'go over' the end of the sequence
                potential_forward_half = sequence[position:position+length]
            else:
                #If it does, break (stop checking)
                break
                
            if len(sequence[position+length:position+(length*2)]) == length:
                potential_reverse_half = sequence[position+length:position+(length*2)]
            else:
                break
            
            if debug:
                print("Position: %i, Length: %i, forward: %s, reverse: %s" % (
                    position + 1,
                    length * 2,
                    potential_forward_half,
                    potential_reverse_half
                )
                     )
            
            
            if potential_forward_half == potential_reverse_half.reverse_complement():
                #This is a palindrome!
                if debug:
                    print("%s and %s appear to form a palindrome!" % (
                    potential_forward_half,
                    potential_reverse_half
                    )
                         )
                
                palindrome_dict[position + 1] = ( #add 1 to compensate for Python's 0-based counting
                    2 * length, #multiply by 2 to give the length of the complete palindrome
                    potential_forward_half + potential_reverse_half
                )
            else:
                pass
            
    return palindrome_dict

In [29]:
example_palindromes = find_DNA_palindromes(example_sequence, debug=True)

Position: 1, Length: 4, forward: TC, reverse: AA
Position: 1, Length: 6, forward: TCA, reverse: ATG
Position: 1, Length: 8, forward: TCAA, reverse: TGCA
Position: 1, Length: 10, forward: TCAAT, reverse: GCATG
Position: 1, Length: 12, forward: TCAATG, reverse: CATGCG
Position: 2, Length: 4, forward: CA, reverse: AT
Position: 2, Length: 6, forward: CAA, reverse: TGC
Position: 2, Length: 8, forward: CAAT, reverse: GCAT
Position: 2, Length: 10, forward: CAATG, reverse: CATGC
Position: 2, Length: 12, forward: CAATGC, reverse: ATGCGG
Position: 3, Length: 4, forward: AA, reverse: TG
Position: 3, Length: 6, forward: AAT, reverse: GCA
Position: 3, Length: 8, forward: AATG, reverse: CATG
Position: 3, Length: 10, forward: AATGC, reverse: ATGCG
Position: 3, Length: 12, forward: AATGCA, reverse: TGCGGG
Position: 4, Length: 4, forward: AT, reverse: GC
Position: 4, Length: 6, forward: ATG, reverse: CAT
ATG and CAT appear to form a palindrome!
Position: 4, Length: 8, forward: ATGC, reverse: ATGC
Posit

In [30]:
example_palindromes = find_DNA_palindromes(example_sequence)

print(example_palindromes)

{4: (6, Seq('ATGCAT', SingleLetterAlphabet())), 5: (4, Seq('TGCA', SingleLetterAlphabet())), 6: (6, Seq('GCATGC', SingleLetterAlphabet())), 7: (4, Seq('CATG', SingleLetterAlphabet())), 17: (4, Seq('TATA', SingleLetterAlphabet())), 18: (4, Seq('ATAT', SingleLetterAlphabet())), 20: (6, Seq('ATGCAT', SingleLetterAlphabet())), 21: (4, Seq('TGCA', SingleLetterAlphabet()))}


Now that this seems to work too, let's include a final function that handles printing in the right format.

In [31]:
def print_palindromes(palindrome_dict):
    """
Given a dictionary with palindrome positions as keys, and
lengths as first element of the value,
print the positions and lengths separated by a whitespace,
one pair per line.
    """
    for key, value in palindrome_dict.items():
        print(key, value[0])
        
    return None

In [32]:
print_palindromes(example_palindromes)

4 6
5 4
6 6
7 4
17 4
18 4
20 6
21 4


With that, I think I have everything covered. Let's try the actual exercise!

In [37]:
test_file = Path("data/rosalind_revp.txt")

(test_name, test_sequence) = read_fasta_file(test_file)

palindromes = find_DNA_palindromes(test_sequence)

print_palindromes(palindromes)

1 4
2 8
3 6
4 4
15 4
27 6
28 4
31 4
38 4
56 4
66 4
76 4
77 4
97 4
110 4
117 4
127 4
131 8
132 6
133 4
136 4
157 4
170 4
193 4
195 4
200 4
220 4
234 4
245 8
246 6
247 4
328 8
329 6
330 4
336 4
343 4
369 4
370 4
428 4
440 4
472 4
480 12
481 10
482 8
483 6
484 4
493 4
508 4
532 4
542 4
570 4
577 6
578 4
602 4
606 4
623 4
638 6
639 4
640 4
647 4
665 4
674 6
675 4
678 4
708 4
716 4
719 4
745 4
780 4
791 6
792 4
800 4
803 4
810 4
814 4
833 6
834 4
841 4
848 4
855 4
859 6
860 4
873 4
890 6
891 4
903 6
904 4
907 4
963 6
964 4
967 4


Apparently, this answer is wrong. Let's check where it may have gone wrong...

In [38]:
print("Length of test sequence: %i" % len(test_sequence))

Length of test sequence: 970


In [39]:
find_mistakes = find_DNA_palindromes(test_sequence, debug = True)

Position: 1, Length: 4, forward: AT, reverse: AT
AT and AT appear to form a palindrome!
Position: 1, Length: 6, forward: ATA, reverse: TTA
Position: 1, Length: 8, forward: ATAT, reverse: TAAT
Position: 1, Length: 10, forward: ATATT, reverse: AATAA
Position: 1, Length: 12, forward: ATATTA, reverse: ATAACG
Position: 2, Length: 4, forward: TA, reverse: TT
Position: 2, Length: 6, forward: TAT, reverse: TAA
Position: 2, Length: 8, forward: TATT, reverse: AATA
TATT and AATA appear to form a palindrome!
Position: 2, Length: 10, forward: TATTA, reverse: ATAAC
Position: 2, Length: 12, forward: TATTAA, reverse: TAACGC
Position: 3, Length: 4, forward: AT, reverse: TA
Position: 3, Length: 6, forward: ATT, reverse: AAT
ATT and AAT appear to form a palindrome!
Position: 3, Length: 8, forward: ATTA, reverse: ATAA
Position: 3, Length: 10, forward: ATTAA, reverse: TAACG
Position: 3, Length: 12, forward: ATTAAT, reverse: AACGCC
Position: 4, Length: 4, forward: TT, reverse: AA
TT and AA appear to form a 

It looks alright at first glance... What may have gone wrong??